### 1. Nhập thư viện và tập dữ liệu

In [1]:
import numpy as np
import pandas as pd
import json
import csv
import random

from tensorflow.keras.preprocessing.text import Tokenizer
from tensorflow.keras.preprocessing.sequence import pad_sequences
from tensorflow.keras.utils import to_categorical
from tensorflow.keras import regularizers

import pprint
import tensorflow.compat.v1 as tf
from tensorflow.python.framework import ops
from sklearn.model_selection import train_test_split
from sklearn import preprocessing
tf.disable_eager_execution()

#Reading the data
data = pd.read_csv("news.csv")
data.head()


,Unnamed: 0,title,text,label
0,8476,You Can Smell Hillary’s Fear,"Daniel Greenfield, a Shillman Journalism Fello...",FAKE
1,10294,Watch The Exact Moment Paul Ryan Committed Pol...,Google Pinterest Digg Linkedin Reddit Stumbleu...,FAKE
2,3608,Kerry to go to Paris in gesture of sympathy,U.S. Secretary of State John F. Kerry said Mon...,REAL
3,10142,Bernie supporters on Twitter erupt in anger ag...,"— Kaydee King (@KaydeeKing) November 9, 2016 T...",FAKE
4,875,The Battle of New York: Why This Primary Matters,It's primary day in New York and front-runners...,REAL


### 2. Bộ dữ liệu tiền xử lý 
Ta thấy kết quả có 1 cột unname => xóa cột này đi

In [3]:
data = data.drop(["Unnamed: 0"], axis=1)
data.head(5)

,title,text,label
0,You Can Smell Hillary’s Fear,"Daniel Greenfield, a Shillman Journalism Fello...",FAKE
1,Watch The Exact Moment Paul Ryan Committed Pol...,Google Pinterest Digg Linkedin Reddit Stumbleu...,FAKE
2,Kerry to go to Paris in gesture of sympathy,U.S. Secretary of State John F. Kerry said Mon...,REAL
3,Bernie supporters on Twitter erupt in anger ag...,"— Kaydee King (@KaydeeKing) November 9, 2016 T...",FAKE
4,The Battle of New York: Why This Primary Matters,It's primary day in New York and front-runners...,REAL


### Mã hóa dữ liệu
Chuyển đổi giá trị cột label chữ thành các giá trị số

In [4]:
# encoding the labels
le = preprocessing.LabelEncoder()
le.fit(data['label'])
data['label'] = le.transform(data['label'])

Cài đặt các biến cần thiết

In [5]:
embedding_dim = 50
max_length = 54
trunc_type = 'post'
padding_type = 'post'
oov_tok = "<OOV>"
training_size = 3000
test_portion = .1


### Tokenization
Quá trình chia một đoạn văn bản lớn thành các đơn vị hoặc các token riêng biệt.  
Ở đây sử dụng các cột riêng lẻ cho 1 cơ sở thời gian như 1 pipeline (chuỗi công việc) để có thể đạt được độ chính xác tốt

In [6]:
title = []
text = []
labels = []
for x in range(training_size):
    title.append(data['title'][x])
    text.append(data['text'][x])
    labels.append(data['label'][x])

Áp dụng TOKENIZATION


In [9]:
tokenizer1 = Tokenizer()
tokenizer1.fit_on_texts(title)
word_index1 = tokenizer1.word_index
vocab_size1 = len(word_index1)
sequences1 = tokenizer1.texts_to_sequences(title)
padded1 = pad_sequences(
    sequences1, padding = padding_type, truncating = trunc_type
)
split = int(test_portion*training_size)
training_sequences1 = padded1[split:training_size]
test_sequences1 = padded1[0:split]
test_labels = labels[0:split]
training_labels = labels[split:training_size]

### 3. Generating Word Embedding
Kỹ thuật nhúng cho phép các từ có ý nghĩa tương tự biểu diễn sẽ tương tự nhau. Ở đây, mỗi từ riêng lẻ được biểu diễn dưới dạng vector có giá trị thực trong một không gian vector đã được xác định trước.  
Trong trường hợp này ta dùng [glove.6B.50d.txt](https://drive.google.com/file/d/1ekbxlI_GdF3H_XHS8U2Csj5q5AhNXhMp/view?usp=sharing) một tệp có chứa các vector nhúng đã được huấn luyện từ trước.  
Glove là một phương pháp nhúng phổ biến được sử dụng trong NLP. Được huấn luyện trên 1 lượng lớn văn bản từ các nguồn khác nhau và mỗi từ trong tệp được biểu diễn bằng 1 vector có kích thước 50 chiều.

In [10]:
embeddings_index = {}
with open('glove.6B.50d.txt') as f:
    for line in f:
        values = line.split()
        word = values[0]
        coefs = np.array(values[1:], dtype='float32')
        embeddings_index[word] = coefs

# Generating embeddings
embedding_matrix = np.zeros((vocab_size1+1, embedding_dim))
for word, i in word_index1.items():
    embedding_vector = embeddings_index.get(word)
    if embedding_vector is not None:
        embedding_matrix[i] = embedding_vector

### 4. Creating Model Architecture
Giờ chúng ta sẽ dùng Tensorflow để tạo mô hình.  
Ở đây ta dùng kỹ thuật TensorFlow embedding với Keras Embedding Layer, lớp này có thể ánh xạ dữ liệu đầu vào ban đầu thành tập hợp các chiều có giá trị thực

In [12]:
model = tf.keras.Sequential([
    tf.keras.layers.Embedding(vocab_size1+1, embedding_dim, input_length = max_length,
                              weights = [embedding_matrix], trainable = False),
    tf.keras.layers.Dropout(0.2),
    tf.keras.layers.Conv1D(64, 5, activation='relu'),
    tf.keras.layers.MaxPooling1D(pool_size=4),
    tf.keras.layers.LSTM(64),
    tf.keras.layers.Dense(1, activation='sigmoid')
])
model.compile( loss = 'binary_crossentropy',
               optimizer='adam', metrics = ['accuracy'])
model.summary()

Model: "sequential_1"
_________________________________________________________________
 Layer (type)                Output Shape              Param #   
 embedding_1 (Embedding)     (None, 54, 50)            377600    
                                                                 
 dropout_1 (Dropout)         (None, 54, 50)            0         
                                                                 
 conv1d_1 (Conv1D)           (None, 50, 64)            16064     
                                                                 
 max_pooling1d_1 (MaxPoolin  (None, 12, 64)            0         
 g1D)                                                            
                                                                 
 lstm_1 (LSTM)               (None, 64)                33024     
                                                                 
 dense_1 (Dense)             (None, 1)                 65        
                                                      

TRAINING

In [13]:
num_epochs = 50

training_padded = np.array(training_sequences1)
training_labels = np.array(training_labels)
testing_padded = np.array(test_sequences1)
testing_labels = np.array(test_labels)

history = model.fit(training_padded, training_labels, epochs = num_epochs,
                    validation_data = (testing_padded, testing_labels), verbose=2)


Train on 2700 samples, validate on 300 samples
Epoch 1/50


c:\Users\Admin\anaconda3\Lib\site-packages\keras\src\engine\training_v1.py:2335: UserWarning: `Model.state_updates` will be removed in a future version. This property should not be used in TensorFlow 2.0, as `updates` are applied automatically.
  updates = self.state_updates


2700/2700 - 3s - loss: 0.6425 - accuracy: 0.5959 - val_loss: 0.5646 - val_accuracy: 0.6733 - 3s/epoch - 1ms/sample
Epoch 2/50
2700/2700 - 1s - loss: 0.5751 - accuracy: 0.7011 - val_loss: 0.5509 - val_accuracy: 0.7033 - 1s/epoch - 508us/sample
Epoch 3/50
2700/2700 - 2s - loss: 0.5439 - accuracy: 0.7233 - val_loss: 0.5116 - val_accuracy: 0.7300 - 2s/epoch - 648us/sample
Epoch 4/50
2700/2700 - 2s - loss: 0.4969 - accuracy: 0.7619 - val_loss: 0.5053 - val_accuracy: 0.7500 - 2s/epoch - 743us/sample
Epoch 5/50
2700/2700 - 2s - loss: 0.4498 - accuracy: 0.7911 - val_loss: 0.5007 - val_accuracy: 0.7300 - 2s/epoch - 599us/sample
Epoch 6/50
2700/2700 - 2s - loss: 0.4067 - accuracy: 0.8070 - val_loss: 0.4715 - val_accuracy: 0.7700 - 2s/epoch - 652us/sample
Epoch 7/50
2700/2700 - 2s - loss: 0.3736 - accuracy: 0.8307 - val_loss: 0.4721 - val_accuracy: 0.7733 - 2s/epoch - 709us/sample
Epoch 8/50
2700/2700 - 2s - loss: 0.3389 - accuracy: 0.8556 - val_loss: 0.4878 - val_accuracy: 0.7833 - 2s/epoch - 76

### 5. Đánh giá và dự đoán mô hình
Thử kiểm tra mô hình bằng một vài văn bản tin tức thông dụng xem nó đúng hay sai


In [18]:
# Sample text to check if fake or not
X = "Duyen is president of America"

#detection
sequences = tokenizer1.texts_to_sequences([X])[0]
sequences = pad_sequences([sequences], maxlen = 54, padding = padding_type,
                          truncating = trunc_type)
if(model.predict(sequences, verbose=0)[0][0]>=0.5):
    print("This news is True")
else:
    print("This news is False")

This news is False
